Count vector 및 TF-IDF vector 추가 → feature로 추가해서 쓰일 수 있을 것 같습니다. 다른 모델에 ensemble로 써도 성능이 괜찮을 것 같아요.

Tutorial Link: https://kgptalkie.com/multi-label-text-classification-on-stack-overflow-tag-prediction/

In [44]:
import pandas as pd
import numpy as np

In [45]:
DATA_PATH = "./data"


In [46]:
# load the preprocessed dataset
df_preprocessed = pd.read_csv(f"{DATA_PATH}/data_preprocessed.csv", encoding="utf-8")
df_preprocessed.head()

,chapter,qtid,text,math_field,hangul_field,tag_field,name,subtitle,qplay_question_id,question_difficulty,question_type_difficulty,grade,purpose,word2vec_pred_chapter,word2vec_pred_name,tfidf_pred_chapter
0,H1S1-08,H1S1-08-05,"연립부등식 `{(-x-3 >= -2x+1), (2x+a <= 10) :}`의 해가 ...","['{(-x-3 >= -2x+1), (2x+a <= 10) :}', 'a']","['연립부등식', '해', '존재하도록', '하', '최댓값', '구하']",[],해를 갖거나 갖지 않는 연립일차부등식,NaN,3504,3,2.0,10,open,"['H1S1-08', 'H1S1-08', 'H1S1-08']","['연립이차부등식', '정수인 해의 개수가 주어진 연립부등식', '해를 갖거나 갖지...",H1S1-08
1,H1S1-08,H1S1-08-11,부등식 `[x]^2-2[x]-3 < 0`의 해는?,['[x]^2-2[x]-3 < 0'],"['부등식', '해']",[],절댓값 또는 가우스 기호가 포함된 이차부등식,NaN,3675,3,2.0,10,open,"['H1S1-08', 'HSU1-06', 'H1S1-08']","['이차부등식', '삼각부등식 (2)', '절댓값 또는 가우스 기호가 포함된 이차부...",H1S1-08
2,H1S1-11,H1S1-11-05,"점 `(1, 2)`를 지나며 중심이 직선 `x-y-1=0` 위에 있고 `x`축에 접...","['(1, 2)', 'x-y-1=0', 'x']","['점', '지나', '중심', '직선', '위', '있', '축', '접하', '...",[],x축 또는 y축에 접하는 원의 방정식,NaN,3779,3,2.0,10,open,"['H1S1-11', 'H1S1-12', 'H1S1-11']","['x축 또는 y축에 접하는 원의 방정식', '평행이동과 대칭이동의 활용', 'x축...",H1S1-11
3,H1S1-07,H1S1-07-18,"`x`, `y`가 정수일 때, 방정식 `xy+2x-3y=7`을 만족하는 순서쌍 `(...","['x', 'y', 'xy+2x-3y=7', '(x, y)']","['정수일', '때', '방정식', '만족하', '순서쌍', '개수']",[],정수 및 자연수 조건을 갖는 부정방정식,NaN,3649,3,3.0,10,open,"['H1S1-07', 'H1S1-10', 'H1S1-08']","['정수 및 자연수 조건을 갖는 부정방정식', '평행한 두 직선 사이의 거리', '...",H1S1-07
4,H1S1-12,H1S1-12-03,"평행이동 `(x, y)->(x+2, y-7)`에 의해 포물선 `y=x^2`을 평행이...","['(x, y)->(x+2, y-7)', 'y=x^2', 'y=x^2+px+q', ...","['평행이동', '의해', '포물선', '평행이동하였더', '포물선', '되었다',...",[],도형의 평행이동 (2),원,3388,2,2.0,10,open,"['H1S1-12', 'H1S1-06', 'H1S2-03']","['도형의 평행이동 (2)', '식의 최댓값과 최솟값 (2)', '코시-슈바르츠의 ...",H1S1-01


In [47]:
# check the type for the label of 중단원: it is a single string label
df_preprocessed["chapter"].iloc[0], type(df_preprocessed["chapter"].iloc[0])

('H1S1-08', str)

In [48]:
# import type setting module
import ast

df_preprocessed['word2vec_pred_chapter'] = df_preprocessed['word2vec_pred_chapter'].apply(lambda x: ast.literal_eval(x))
df_preprocessed.head()



,chapter,qtid,text,math_field,hangul_field,tag_field,name,subtitle,qplay_question_id,question_difficulty,question_type_difficulty,grade,purpose,word2vec_pred_chapter,word2vec_pred_name,tfidf_pred_chapter
0,H1S1-08,H1S1-08-05,"연립부등식 `{(-x-3 >= -2x+1), (2x+a <= 10) :}`의 해가 ...","['{(-x-3 >= -2x+1), (2x+a <= 10) :}', 'a']","['연립부등식', '해', '존재하도록', '하', '최댓값', '구하']",[],해를 갖거나 갖지 않는 연립일차부등식,NaN,3504,3,2.0,10,open,"[H1S1-08, H1S1-08, H1S1-08]","['연립이차부등식', '정수인 해의 개수가 주어진 연립부등식', '해를 갖거나 갖지...",H1S1-08
1,H1S1-08,H1S1-08-11,부등식 `[x]^2-2[x]-3 < 0`의 해는?,['[x]^2-2[x]-3 < 0'],"['부등식', '해']",[],절댓값 또는 가우스 기호가 포함된 이차부등식,NaN,3675,3,2.0,10,open,"[H1S1-08, HSU1-06, H1S1-08]","['이차부등식', '삼각부등식 (2)', '절댓값 또는 가우스 기호가 포함된 이차부...",H1S1-08
2,H1S1-11,H1S1-11-05,"점 `(1, 2)`를 지나며 중심이 직선 `x-y-1=0` 위에 있고 `x`축에 접...","['(1, 2)', 'x-y-1=0', 'x']","['점', '지나', '중심', '직선', '위', '있', '축', '접하', '...",[],x축 또는 y축에 접하는 원의 방정식,NaN,3779,3,2.0,10,open,"[H1S1-11, H1S1-12, H1S1-11]","['x축 또는 y축에 접하는 원의 방정식', '평행이동과 대칭이동의 활용', 'x축...",H1S1-11
3,H1S1-07,H1S1-07-18,"`x`, `y`가 정수일 때, 방정식 `xy+2x-3y=7`을 만족하는 순서쌍 `(...","['x', 'y', 'xy+2x-3y=7', '(x, y)']","['정수일', '때', '방정식', '만족하', '순서쌍', '개수']",[],정수 및 자연수 조건을 갖는 부정방정식,NaN,3649,3,3.0,10,open,"[H1S1-07, H1S1-10, H1S1-08]","['정수 및 자연수 조건을 갖는 부정방정식', '평행한 두 직선 사이의 거리', '...",H1S1-07
4,H1S1-12,H1S1-12-03,"평행이동 `(x, y)->(x+2, y-7)`에 의해 포물선 `y=x^2`을 평행이...","['(x, y)->(x+2, y-7)', 'y=x^2', 'y=x^2+px+q', ...","['평행이동', '의해', '포물선', '평행이동하였더', '포물선', '되었다',...",[],도형의 평행이동 (2),원,3388,2,2.0,10,open,"[H1S1-12, H1S1-06, H1S2-03]","['도형의 평행이동 (2)', '식의 최댓값과 최솟값 (2)', '코시-슈바르츠의 ...",H1S1-01


In [49]:
from sklearn.preprocessing import LabelBinarizer

# Binarizing single label 중단원
single_label = LabelBinarizer()
y = single_label.fit_transform(df_preprocessed['chapter'])
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [50]:
# check classes for the single label
single_label.classes_

array(['H1S1-01', 'H1S1-02', 'H1S1-03', 'H1S1-04', 'H1S1-05', 'H1S1-06',
       'H1S1-07', 'H1S1-08', 'H1S1-09', 'H1S1-10', 'H1S1-11', 'H1S1-12',
       'H1S2-01', 'H1S2-02', 'H1S2-03', 'H1S2-04', 'H1S2-05', 'H1S2-06',
       'H1S2-07', 'HSTA-01', 'HSTA-02', 'HSTA-03', 'HSTA-04', 'HSTA-05',
       'HSTA-06', 'HSTA-07', 'HSU1-01', 'HSU1-02', 'HSU1-03', 'HSU1-04',
       'HSU1-05', 'HSU1-06', 'HSU1-07', 'HSU1-08', 'HSU1-09', 'HSU1-10',
       'HSU1-11'], dtype='<U7')

In [51]:
pd.DataFrame(y, columns=single_label.classes_)


,H1S1-01,H1S1-02,H1S1-03,H1S1-04,H1S1-05,H1S1-06,H1S1-07,H1S1-08,H1S1-09,H1S1-10,...,HSU1-02,HSU1-03,HSU1-04,HSU1-05,HSU1-06,HSU1-07,HSU1-08,HSU1-09,HSU1-10,HSU1-11
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5359,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5360,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5361,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5362,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
# Reading josa from txt file
list_josa_read = []

# open file and read the content in a list
with open(f'{DATA_PATH}/list_josa.txt', 'rt', encoding="UTF8") as input_file:
    for line in input_file:
        # remove linebreak which is the last character of the string
        str_josa = line[:-1]

        # add item to the list
        list_josa_read.append(str_josa)
print(len(list_josa_read),list_josa_read)

84 ['가', '이다', '이라', '까지는', '으로부터의', '에도', '나', '까지의', '여', '하고', '과', '만이', '이', '에서', '와는', '에서도', '이라서', '만', '여서', '서', '이며', '야', '에서부터', '로', '에서는', '이어서', '니', '부터는', '인', '인지', '은', '만큼', '네', '라', '의', '도', '에서만', '과의', '다', '이야', '부터', '에는', '에게', '일수록', '인가', '으로의', '까지', '이지만', '이고', '며', '한', '만에', '으로', '만의', '를', '고', '이라는', '래', '을', '면', '라도', '이나', '으로만', '와의', '아', '보다', '만을', '대로', '으로부터', '이네', '지', '라고', '와', '이라고', '에서의', '이를', '는', '에', '과는', '라는', '마다', '에만', '만큼씩', '에게는']


In [53]:
non_math_terms = ['<br/>', '<fieldset>', '</fieldset>', '<legend>', '</legend>',"<span>","</span>"]


In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

list_stopwords = list_josa_read + non_math_terms

# TFIDF vectorizing the question text
tfidf = TfidfVectorizer(analyzer='word', max_features=10000, ngram_range=(1,3), stop_words=list_stopwords)
X = tfidf.fit_transform(df_preprocessed["text"])
X.shape, y.shape

((5364, 10000), (5364, 37))

In [55]:
dict_vocab = tfidf.vocabulary_
list(dict_vocab)[:20]

['연립부등식',
 '2x',
 '10',
 '해가',
 '존재하도록',
 '하는',
 '최댓값을',
 '구하면',
 '연립부등식 2x',
 '2x 2x',
 '2x 10',
 '해가 존재하도록',
 '존재하도록 하는',
 '하는 최댓값을',
 '최댓값을 구하면',
 '해가 존재하도록 하는',
 '부등식',
 '해는',
 '부등식 해는',
 '중심이']

In [56]:
from sklearn.model_selection import train_test_split

# splitting train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [57]:
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import LinearSVC
# load sklearn's classifiers

sgd = SGDClassifier()
lr = LogisticRegression(solver='lbfgs')
svc = LinearSVC()


In [58]:
def j_score(y_true, y_pred):
  jaccard = np.minimum(y_true, y_pred).sum(axis = 1)/np.maximum(y_true, y_pred).sum(axis = 1)
  return jaccard.mean()*100


def print_score(y_pred, clf):
  print("Clf: ", clf.__class__.__name__)
  print('Jacard score: {}'.format(j_score(y_test, y_pred)))
  print('----')

In [59]:
from sklearn.multiclass import OneVsRestClassifier

for classifier in [LinearSVC(C=1.5, penalty = 'l1', dual=False)]:
  clf = OneVsRestClassifier(classifier)
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print_score(y_pred, classifier)


Clf:  LinearSVC
Jacard score: 63.886300093196645
----


In [60]:
for classifier in [sgd, lr, svc]:
  clf = OneVsRestClassifier(classifier)
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print_score(y_pred, classifier)


Clf:  SGDClassifier
Jacard score: 67.61416589002795
----
Clf:  LogisticRegression
Jacard score: 18.732525629077355
----
Clf:  LinearSVC
Jacard score: 65.37744641192917
----


## Labeling chapter label prediction with tfidf model

In [61]:
clf = OneVsRestClassifier(sgd)
clf.fit(X_train, y_train)

x = [ '이차함수 `y=(x-3)^2+6`을 `x`축에 대하여 대칭이동한 도형의 방정식은?']
xt = tfidf.transform(x)
clf.predict(xt)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [62]:
single_label.inverse_transform(clf.predict(xt))

array(['H1S1-12'], dtype='<U7')

In [63]:
array_x = df_preprocessed["text"]
array_x_test = tfidf.transform(array_x)
array_x_test

<5364x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 111660 stored elements in Compressed Sparse Row format>

In [64]:
print(len(single_label.inverse_transform(clf.predict(array_x_test))))
single_label.inverse_transform(clf.predict(array_x_test))

5364


array(['H1S1-08', 'H1S1-08', 'H1S1-11', ..., 'H1S2-04', 'H1S2-03',
       'H1S2-07'], dtype='<U7')

In [65]:
df_preprocessed["tfidf_pred_chapter"] = single_label.inverse_transform(clf.predict(array_x_test))
df_preprocessed.head()

,chapter,qtid,text,math_field,hangul_field,tag_field,name,subtitle,qplay_question_id,question_difficulty,question_type_difficulty,grade,purpose,word2vec_pred_chapter,word2vec_pred_name,tfidf_pred_chapter
0,H1S1-08,H1S1-08-05,"연립부등식 `{(-x-3 >= -2x+1), (2x+a <= 10) :}`의 해가 ...","['{(-x-3 >= -2x+1), (2x+a <= 10) :}', 'a']","['연립부등식', '해', '존재하도록', '하', '최댓값', '구하']",[],해를 갖거나 갖지 않는 연립일차부등식,NaN,3504,3,2.0,10,open,"[H1S1-08, H1S1-08, H1S1-08]","['연립이차부등식', '정수인 해의 개수가 주어진 연립부등식', '해를 갖거나 갖지...",H1S1-08
1,H1S1-08,H1S1-08-11,부등식 `[x]^2-2[x]-3 < 0`의 해는?,['[x]^2-2[x]-3 < 0'],"['부등식', '해']",[],절댓값 또는 가우스 기호가 포함된 이차부등식,NaN,3675,3,2.0,10,open,"[H1S1-08, HSU1-06, H1S1-08]","['이차부등식', '삼각부등식 (2)', '절댓값 또는 가우스 기호가 포함된 이차부...",H1S1-08
2,H1S1-11,H1S1-11-05,"점 `(1, 2)`를 지나며 중심이 직선 `x-y-1=0` 위에 있고 `x`축에 접...","['(1, 2)', 'x-y-1=0', 'x']","['점', '지나', '중심', '직선', '위', '있', '축', '접하', '...",[],x축 또는 y축에 접하는 원의 방정식,NaN,3779,3,2.0,10,open,"[H1S1-11, H1S1-12, H1S1-11]","['x축 또는 y축에 접하는 원의 방정식', '평행이동과 대칭이동의 활용', 'x축...",H1S1-11
3,H1S1-07,H1S1-07-18,"`x`, `y`가 정수일 때, 방정식 `xy+2x-3y=7`을 만족하는 순서쌍 `(...","['x', 'y', 'xy+2x-3y=7', '(x, y)']","['정수일', '때', '방정식', '만족하', '순서쌍', '개수']",[],정수 및 자연수 조건을 갖는 부정방정식,NaN,3649,3,3.0,10,open,"[H1S1-07, H1S1-10, H1S1-08]","['정수 및 자연수 조건을 갖는 부정방정식', '평행한 두 직선 사이의 거리', '...",H1S1-07
4,H1S1-12,H1S1-12-03,"평행이동 `(x, y)->(x+2, y-7)`에 의해 포물선 `y=x^2`을 평행이...","['(x, y)->(x+2, y-7)', 'y=x^2', 'y=x^2+px+q', ...","['평행이동', '의해', '포물선', '평행이동하였더', '포물선', '되었다',...",[],도형의 평행이동 (2),원,3388,2,2.0,10,open,"[H1S1-12, H1S1-06, H1S2-03]","['도형의 평행이동 (2)', '식의 최댓값과 최솟값 (2)', '코시-슈바르츠의 ...",H1S1-01


In [66]:
# df_preprocessed.to_csv(f"{DATA_PATH}/data_preprocessed.csv", index=False, encoding="UTF-8")

## test for joint tfidf - word2vec model on training dataset

In [72]:
list_combined = []
for index, row in df_preprocessed.iterrows():
    if row["tfidf_pred_chapter"] in row["word2vec_pred_chapter"]:
        list_combined.append(row["tfidf_pred_chapter"])
    else: 
        list_combined.append(row["word2vec_pred_chapter"][0])
list_combined[:4]

['H1S1-08', 'H1S1-08', 'H1S1-11', 'H1S1-07']

In [73]:
df_temp = df_preprocessed.copy()

In [74]:
df_temp["combined"] = list_combined
df_temp.head()

,chapter,qtid,text,math_field,hangul_field,tag_field,name,subtitle,qplay_question_id,question_difficulty,question_type_difficulty,grade,purpose,word2vec_pred_chapter,word2vec_pred_name,tfidf_pred_chapter,combined
0,H1S1-08,H1S1-08-05,"연립부등식 `{(-x-3 >= -2x+1), (2x+a <= 10) :}`의 해가 ...","['{(-x-3 >= -2x+1), (2x+a <= 10) :}', 'a']","['연립부등식', '해', '존재하도록', '하', '최댓값', '구하']",[],해를 갖거나 갖지 않는 연립일차부등식,NaN,3504,3,2.0,10,open,"[H1S1-08, H1S1-08, H1S1-08]","['연립이차부등식', '정수인 해의 개수가 주어진 연립부등식', '해를 갖거나 갖지...",H1S1-08,H1S1-08
1,H1S1-08,H1S1-08-11,부등식 `[x]^2-2[x]-3 < 0`의 해는?,['[x]^2-2[x]-3 < 0'],"['부등식', '해']",[],절댓값 또는 가우스 기호가 포함된 이차부등식,NaN,3675,3,2.0,10,open,"[H1S1-08, HSU1-06, H1S1-08]","['이차부등식', '삼각부등식 (2)', '절댓값 또는 가우스 기호가 포함된 이차부...",H1S1-08,H1S1-08
2,H1S1-11,H1S1-11-05,"점 `(1, 2)`를 지나며 중심이 직선 `x-y-1=0` 위에 있고 `x`축에 접...","['(1, 2)', 'x-y-1=0', 'x']","['점', '지나', '중심', '직선', '위', '있', '축', '접하', '...",[],x축 또는 y축에 접하는 원의 방정식,NaN,3779,3,2.0,10,open,"[H1S1-11, H1S1-12, H1S1-11]","['x축 또는 y축에 접하는 원의 방정식', '평행이동과 대칭이동의 활용', 'x축...",H1S1-11,H1S1-11
3,H1S1-07,H1S1-07-18,"`x`, `y`가 정수일 때, 방정식 `xy+2x-3y=7`을 만족하는 순서쌍 `(...","['x', 'y', 'xy+2x-3y=7', '(x, y)']","['정수일', '때', '방정식', '만족하', '순서쌍', '개수']",[],정수 및 자연수 조건을 갖는 부정방정식,NaN,3649,3,3.0,10,open,"[H1S1-07, H1S1-10, H1S1-08]","['정수 및 자연수 조건을 갖는 부정방정식', '평행한 두 직선 사이의 거리', '...",H1S1-07,H1S1-07
4,H1S1-12,H1S1-12-03,"평행이동 `(x, y)->(x+2, y-7)`에 의해 포물선 `y=x^2`을 평행이...","['(x, y)->(x+2, y-7)', 'y=x^2', 'y=x^2+px+q', ...","['평행이동', '의해', '포물선', '평행이동하였더', '포물선', '되었다',...",[],도형의 평행이동 (2),원,3388,2,2.0,10,open,"[H1S1-12, H1S1-06, H1S2-03]","['도형의 평행이동 (2)', '식의 최댓값과 최솟값 (2)', '코시-슈바르츠의 ...",H1S1-01,H1S1-12


In [78]:
df_temp["chapter"].compare(df_temp["combined"])
1 - len(df_temp["chapter"].compare(df_temp["combined"]))/len(df_temp)

0.7900820283370619

In [79]:
print(1 - len(df_temp["chapter"].compare(df_temp["tfidf_pred_chapter"]))/len(df_temp))

0.8922445935868755


## predict for evaluation dataset

In [68]:
df_test = pd.read_csv(DATA_PATH+'/hidden_for_inference.csv')
df_test.head(2)

,qplay_question_id,text
0,3535,"좌표평면 위의 두 점 `A(1, 2)`, `B(5, -2)`에 대하여 선분 `AB`..."
1,4537,"`a-b=3`일 때, `a^3-3a^2b+(3b^2-5)a-b^3+5b-10`의 값은?"


In [69]:
array_x_eval = df_test["text"]
array_x_test_eval = tfidf.transform(array_x_eval)
array_x_test_eval
print(len(single_label.inverse_transform(clf.predict(array_x_test_eval))))
single_label.inverse_transform(clf.predict(array_x_test_eval))

1378


array(['H1S1-09', 'H1S1-01', 'H1S1-08', ..., 'H1S2-07', 'H1S2-06',
       'H1S2-04'], dtype='<U7')

In [70]:
df_test["tfidf_pred_chapter"] = single_label.inverse_transform(clf.predict(array_x_test_eval))
df_test.head()

,qplay_question_id,text,tfidf_pred_chapter
0,3535,"좌표평면 위의 두 점 `A(1, 2)`, `B(5, -2)`에 대하여 선분 `AB`...",H1S1-09
1,4537,"`a-b=3`일 때, `a^3-3a^2b+(3b^2-5)a-b^3+5b-10`의 값은?",H1S1-01
2,3910,`-2 <= x <= 3`인 모든 실수 `x`에 대하여 이차부등식 `-x^2-2x+...,H1S1-08
3,4294,"네 점 `A(0, a)`, `B(b, 1)`, `C(5, 2)`, `D(4, 4)`...",H1S1-09
4,4319,"다항식 `x^3+x^2`을 다항식 `x^2-1`로 나누었을 때의 몫을 `Q(x)`,...",H1S1-01


In [71]:
# df_test.to_csv("./data/data_eval_tfidf.csv", index=False, encoding="utf-8")